In [1]:
!pip install ultralytics
!pip install opencv-python
!pip install matplotlib
!pip install PyYAML

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:00


In [2]:
import os
import yaml
import shutil
from ultralytics import YOLO
from pathlib import Path

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import os
from ultralytics import YOLO

PROJECT_DIR = "/content/yolo_train"
os.makedirs(PROJECT_DIR, exist_ok=True)

In [5]:
!unzip -q "/content/drive/MyDrive/hackwithhyderabad/hackathon2_train_1.zip" -d /content/

In [6]:
yaml_text = """
train: /content/train_1/train1/images
val: /content/train_1/val1/images
nc: 7
names: ['OxygenTank', 'NitrogenTank', 'FirstAidBox', 'FireAlarm', 'SafetySwitchPanel', 'EmergencyPhone', 'FireExtinguisher']
"""
with open("/content/train_1/data.yaml", "w") as f:
    f.write(yaml_text)

print("data.yaml created at /content/train_1/data.yaml")

data.yaml created at /content/train_1/data.yaml


In [7]:
model = YOLO("yolov8s.pt")
print("YOLOv8s model loaded.")

YOLOv8s model loaded.


In [8]:
results = model.train(
    data="/content/train_1/data.yaml",
    epochs=20,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=1e-4,
    lrf=1e-4,
    momentum=0.9,
    mosaic=0.4,
    project="yolo_train",  # safe project name for wandb
    name="exp_yolo",
    save=True,
    save_period=-1,
    device=0
)
print("Training complete. Results saved in Google Drive folder:", PROJECT_DIR)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/train_1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.9, mosaic=0.4, multi_scale=False, name=exp_yolo, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

In [9]:
metrics = model.val()
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)
print("Precision:", metrics.box.mp)
print("Recall:", metrics.box.mr)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3129.6±962.7 MB/s, size: 2822.8 KB)
val: Scanning /content/train_1/val1/labels.cache... 336 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 336/336 411.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 21/21 1.1it/s 19.1s
                   all        336       1005      0.884      0.665      0.744      0.613
            OxygenTank        156        257      0.901      0.732      0.809      0.718
          NitrogenTank        165        300      0.911      0.747      0.824      0.749
           FirstAidBox         82        121       0.86      0.702      0.758      0.649
             FireAlarm         54         60      0.897      0.567      0.705      0.562
     SafetySwitchPanel         53         5

In [10]:
eval_dir = "/content/train_1/val1/images"
results = model.predict(source=eval_dir, save=True, conf=0.25)
print("Predictions saved at:", results[0].save_dir)


image 1/336 /content/train_1/val1/images/000000000_vcluttered_hallway.png: 384x640 2 EmergencyPhones, 48.2ms
image 2/336 /content/train_1/val1/images/000000001_vlight_cluttered_room.png: 384x640 2 FireAlarms, 1 SafetySwitchPanel, 1 EmergencyPhone, 10.7ms
image 3/336 /content/train_1/val1/images/000000002_vlight_uncluttered.png: 384x640 1 OxygenTank, 10.7ms
image 4/336 /content/train_1/val1/images/000000003_vlight_cluttered.png: 384x640 1 OxygenTank, 10.7ms
image 5/336 /content/train_1/val1/images/000000004_vlight_cluttered.png: 384x640 1 FireExtinguisher, 10.7ms
image 6/336 /content/train_1/val1/images/000000004_vlight_uncluttered.png: 384x640 1 OxygenTank, 3 NitrogenTanks, 10.7ms
image 7/336 /content/train_1/val1/images/000000005_vcluttered_room.png: 384x640 2 NitrogenTanks, 1 FireExtinguisher, 10.7ms
image 8/336 /content/train_1/val1/images/000000006_vlight_uncluttered.png: 384x640 1 OxygenTank, 10.7ms
image 9/336 /content/train_1/val1/images/000000007_vlight_cluttered.png: 384x640 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
